<a href="https://colab.research.google.com/github/MrYeeOus/DIVE-AIEP/blob/main/notebooks/json2dae_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install datasets
!pip install git+https://github.com/huggingface/transformers.git
!pip install accelerate

import ipywidgets as widgets
import pandas as pd
import json
import torch

from datasets import load_dataset

  Created wheel for transformers: filename=transformers-4.41.0.dev0-py3-none-any.whl size=9050615 sha256=bebccceaee4eca45a1128c2941a3fe7a29f61eaa12ffd008a225aa792f070448
  Stored in directory: /tmp/pip-ephem-wheel-cache-mpb7ft6c/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.1
    Uninstalling transformers-4.40.1:
      Successfully uninstalled transformers-4.40.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 2.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1

In [5]:
# GOOGLE COLAB
# Upload the mergedJson.json zip file, to preserve contents
!unzip /content/mergedJson.zip
!sha1sum /content/mergedJson.json
# Should be 12d88910c17f2e7592f6cb9005bf86cdb557e031

Archive:  /content/mergedJson.zip
  inflating: mergedJson.json         
12d88910c17f2e7592f6cb9005bf86cdb557e031  /content/mergedJson.json


In [6]:
with open("/content/mergedJson.json", "rb") as jsonl_file:
    data_list = json.load(jsonl_file)

In [ ]:
# LOCAL
with open("./content/mergedJson.json", "rb") as jsonl_file:
    data_list = json.load(jsonl_file)

In [7]:
from datasets import Dataset
from datasets.dataset_dict import DatasetDict

del data_list["GENERATED_DATA"][len(data_list["GENERATED_DESCRIPTION"]):]

len(data_list["GENERATED_DATA"])


1248

In [8]:
data_lengths = [len(x) for x in data_list["GENERATED_DATA"]]
descript_lengths = [len(x) for x in data_list["GENERATED_DESCRIPTION"]]
print("Data: " + str(max(data_lengths)) + ", " + "Description: " + str(max(descript_lengths)))

Data: 2769, Description: 98


In [9]:
dataframe = pd.DataFrame(data_list, columns=["GENERATED_DESCRIPTION", "GENERATED_DATA"])
dataframe

,GENERATED_DESCRIPTION,GENERATED_DATA
0,"{'location': '<Vector (0.0000, 0.0000, 12.5000...","b'<library_geometries>\n <geometry id=""Obje..."
1,"{'location': '<Vector (-4.0000, -4.0000, 0.500...","b'<library_geometries>\n <geometry id=""Obje..."
2,"{'location': '<Vector (-8.0000, -2.0000, 0.500...","b'<library_geometries>\n <geometry id=""room..."
3,"{'location': '<Vector (9.0000, 0.0000, 0.5000)...","b'<library_geometries>\n <geometry id=""Obje..."
4,"{'location': '<Vector (6.0000, 6.0000, 0.5000)...","b'<library_geometries>\n <geometry id=""Obje..."
...,...,...
1243,"{'location': '<Vector (-1.5000, 5.5000, 0.5000...","b'<library_geometries>\n <geometry id=""Obje..."
1244,"{'location': '<Vector (2.5000, -6.5000, 0.5000...","b'<library_geometries>\n <geometry id=""room..."
1245,"{'location': '<Vector (3.5000, 9.5000, 0.5000)...","b'<library_geometries>\n <geometry id=""Obje..."
1246,"{'location': '<Vector (0.5000, -9.5000, 0.5000...","b'<library_geometries>\n <geometry id=""Obje..."


In [ ]:

dataframe_train = Dataset.from_pandas(dataframe[:600])
dataframe_validate = Dataset.from_pandas(dataframe[600:800])
dataframe_test = Dataset.from_pandas(dataframe[800:])

dataset3 = DatasetDict({"train": dataframe_train, "validate": dataframe_validate, "test": dataframe_test})
print(dataset3)

In [ ]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-small")

max_source_length = 512
max_target_length = 4096

# task_prefix = "json2dae: "


In [ ]:
def preprocess_data(inputData):
    input_sequences = inputData["GENERATED_DESCRIPTION"]
    input_sequences = [str(seq) for seq in input_sequences]

    output_sequences = inputData["GENERATED_DATA"]
    output_sequences = [str(seq) for seq in output_sequences]

    model_inputs = tokenizer(
        input_sequences,
        max_length = max_source_length,
        padding = "max_length",
        truncation = True
    )

    labels = tokenizer(
        output_sequences,
        max_length = max_target_length,
        padding = "max_length",
        truncation = True
    )

    # Replace padding tokens with -100 so they are
    # ignored by CrossEntropyLoss thing
    #labels_with_ignore_index = []
    #for sample in labels:
    #    sample = [label if label != 0 else -100 for label in sample]
    #    labels_with_ignore_index.append(sample)

    labels_with_ignore_index = []
    for label_sample in labels["input_ids"]:
        label_sample = [label if label != 0 else -100 for label in label_sample]
        labels_with_ignore_index.append(label_sample)

    model_inputs["labels"] = labels_with_ignore_index
    return model_inputs



mapped_dataset = dataset3.map(preprocess_data, batched=True, batch_size=2)
mapped_dataset

In [ ]:
mapped_dataset["test"]["labels"][0]

In [ ]:
import pickle

with open('6May24_v1_m512_M512_S1188_mapped_dataset.pkl', 'wb') as f:
    pickle.dump(mapped_dataset, f)

!zip 6May24_v1_m512_M512_S1188_mapped_dataset.pkl.zip 6May24_v1_m512_M512_S1188_mapped_dataset.pkl

In [ ]:
from torch.utils.data import DataLoader
mapped_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
train_dataloader = DataLoader(mapped_dataset["train"], shuffle=True, batch_size=2)
valid_dataloader = DataLoader(mapped_dataset["validate"], batch_size=2)
test_dataloader = DataLoader(mapped_dataset["test"], batch_size=2)

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)




In [ ]:
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

In [ ]:
import torch

from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, DefaultDataCollator

In [ ]:
data_collator = DefaultDataCollator(return_tensors="tf")

mapped_train = mapped_dataset["train"]
mapped_validate = mapped_dataset["validate"]

In [ ]:
dataset_training = mapped_train.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=True,
    batch_size=2,
    collate_fn=data_collator,
)
dataset_validation = mapped_validate.to_tf_dataset(
    columns=["input_ids", "attention_mask", "labels"],
    shuffle=False,
    batch_size=2,
    collate_fn=data_collator,
)

In [ ]:
training_args = TrainingArguments(
    output_dir = "/",
    num_train_epochs = 3,
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    save_steps = 400,
    save_total_limit = 2,
    eval_steps = 100,
    eval_strategy = "steps",
    fp16 = True
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = mapped_dataset["train"],
    eval_dataset = mapped_dataset["validate"]
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("6May24_v1")
!zip -r 6May24_model_v1.zip 6May24_v1

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model_path = "6May24_v1"
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')

In [ ]:
# @title Testing

def translator(model, tokenizer):
  inputs = """
        {'location': '<Vector (0.0000, 0.0000, 9.0000)>', 'size': '<Vector (10.0000, 13.0000, 18.0000)>'}

  """
  input_tokens = tokenizer.encode(
      inputs,
      return_tensors = "pt",
      max_length = 512,
      truncation = True
  )

  corrected_ids = model.generate(
      input_tokens,
      max_length=4096,
      early_stopping=False
  )

  corrected_text = tokenizer.decode(corrected_ids[0], skip_special_tokens = True)
  return corrected_text

tokenstuff = translator(model, tokenizer)
print(tokenstuff)

In [ ]:
tokenstuff

In [ ]:
tokenstuff